In [12]:
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from numpy import dstack
from keras.utils import np_utils
import time
import numpy as np
import random
import pickle
from sklearn.model_selection import train_test_split

In [14]:
# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'models/model_CNN_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX_M,inputX_G):
    stackX = None
    for i in range(len(members)):
        model=members[i]
        if i<7:
            # make prediction
            yhat = model.predict(inputX_M, verbose=0)
            # stack predictions into [rows, members, probabilities]
            if stackX is None:
                stackX = yhat
            else:
                stackX = dstack((stackX, yhat))
        else:
            # make prediction
            yhat = model.predict(inputX_G, verbose=0)
            # stack predictions into [rows, members, probabilities]
            if stackX is None:
                stackX = yhat
            else:
                stackX = dstack((stackX, yhat))            
                
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    inputX_M=inputX[:, :, :, :4]
    inputX_G=inputX[:, :, :, 4:]
    stackedX = stacked_dataset(members, inputX_M,inputX_G)
    # fit standalone model
    model = LogisticRegression(max_iter=6500)
    model.fit(stackedX, inputy)
    return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    inputX_M=inputX[:, :, :, :4]
    inputX_G=inputX[:, :, :, 4:]
    stackedX = stacked_dataset(members, inputX_M,inputX_G)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat


start_time = time.clock()
np.random.seed(7)
random.seed(7)

filename = 'Combined Trajectory_Label_Geolife/Revised_KerasData_Smoothing_GIS_new_8.pickle'

with open(filename, mode='rb') as f:
    TotalInput, FinalLabel = pickle.load(f, encoding='latin1')  # Also can use the encoding 'iso-8859-1'

NoClass = len(list(set(np.ndarray.flatten(FinalLabel))))
Threshold = len(TotalInput[0, 0, :, 0])

# Making training and test data: 80% Training, 20% Test
Train_X, Test_X, Train_Y, Test_Y_ori = train_test_split(TotalInput, FinalLabel, test_size=0.20, random_state=7)

Test_M_X=Test_X[:, :, :, :4]
Test_G_X=Test_X[:, :, :, 4:]


# load all models
n_members = 14
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
for i in range(len(members)):
    model=members[i]
    if i <7:
        testy_enc = np_utils.to_categorical(Test_Y_ori, num_classes=NoClass)
        _, acc = model.evaluate(Test_M_X, testy_enc, verbose=0)
        print('Model Accuracy: %.3f' % acc)
    else:
        testy_enc = np_utils.to_categorical(Test_Y_ori, num_classes=NoClass)
        _, acc = model.evaluate(Test_G_X, testy_enc, verbose=0)
        print('Model Accuracy: %.3f' % acc)
# fit stacked model using the ensemble
model = fit_stacked_model(members, Test_X, Test_Y_ori)
# evaluate model on test set
yhat = stacked_prediction(members, model, Test_X)
acc = accuracy_score(Test_Y_ori, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


>loaded models/model_CNN_1.h5
>loaded models/model_CNN_2.h5
>loaded models/model_CNN_3.h5
>loaded models/model_CNN_4.h5
>loaded models/model_CNN_5.h5
>loaded models/model_CNN_6.h5
>loaded models/model_CNN_7.h5
>loaded models/model_CNN_8.h5
>loaded models/model_CNN_9.h5
>loaded models/model_CNN_10.h5
>loaded models/model_CNN_11.h5
>loaded models/model_CNN_12.h5
>loaded models/model_CNN_13.h5
>loaded models/model_CNN_14.h5
Loaded 14 models
Model Accuracy: 0.801
Model Accuracy: 0.801
Model Accuracy: 0.804
Model Accuracy: 0.815
Model Accuracy: 0.814
Model Accuracy: 0.815
Model Accuracy: 0.815
Model Accuracy: 0.568
Model Accuracy: 0.613
Model Accuracy: 0.630
Model Accuracy: 0.668
Model Accuracy: 0.623
Model Accuracy: 0.673
Model Accuracy: 0.679
Stacked Test Accuracy: 0.836


In [6]:
from sklearn.metrics import confusion_matrix
import time
start_time = time.clock()
NoClass=5
counter = 0
for i in range(len(yhat)):
    if yhat[i] == Test_Y_ori[i]:
        counter += 1
Accuracy = counter * 1./len(yhat)

ActualPositive = []
for i in range(NoClass):
    AA = np.where(Test_Y_ori == i)[0]
    ActualPositive.append(AA)

PredictedPositive = []
for i in range(NoClass):
    AA = np.where(yhat == i)[0]
    PredictedPositive.append(AA)

TruePositive = []
FalsePositive = []
for i in range(NoClass):
    AA = []
    BB = []
    for j in PredictedPositive[i]:
        if yhat[j] == Test_Y_ori[j]:
            AA.append(j)
        else:
            BB.append(j)
    TruePositive.append(AA)
    FalsePositive.append(BB)
Precision = []
Recall = []
for i in range(NoClass):
    Precision.append(len(TruePositive[i]) * 1./len(PredictedPositive[i]))
    Recall.append(len(TruePositive[i]) * 1./len(ActualPositive[i]))

ConfusionM = confusion_matrix(list(Test_Y_ori), yhat, labels=[0, 1, 2, 3, 4])

print('Confusion Matrix: ', ConfusionM)
print("Recall", Recall)
print('precision', Precision)
print(time.clock() - start_time, "seconds")

Confusion Matrix:  [[1988   69   56    4    5]
 [ 169  911   31    9    1]
 [ 133   31 1136  120   19]
 [  73   28  170  594   23]
 [  84    7   15   19  793]]
Recall [0.9368520263901979, 0.8126672613737734, 0.7894371091035441, 0.668918918918919, 0.8638344226579521]
precision [0.8124233755619126, 0.8709369024856597, 0.8068181818181818, 0.7962466487935657, 0.9429250891795482]
0.02159000000000333 seconds


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [53]:
sum(Recall)/len(Recall)


0.6386215160483345

In [54]:
sum(Precision)/len(Precision)

0.6805524141188277

In [55]:
F_score=[]
for i in range(len(Precision)):
    f=(2*Precision[i]*Recall[i])/(Precision[i]+Recall[i])
    F_score.append(f)

In [56]:
sum(F_score)/len(F_score)

0.6469828061229403